In [1]:
import torch
import pandas as pd
import numpy as np

c:\Users\chang\Anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


#### 1. 데이터구성

In [3]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):    
    # 객체생성시 한번실행
    def __init__(self, data_path, transform=None, target_transform=None):
        
        data = pd.read_csv(data_path)
        Y_data = data['label']
        Y_data = np.array(Y_data)
        X_data = data.drop(columns='label',axis=1)
        X_data = np.array(X_data).reshape(-1,28,28,1).astype('float32')
        self.X_data = X_data
        self.Y_data = Y_data
        self.transform = transform
        self.target_transform = target_transform

    # 데이터세트의 총 개수를 리턴
    def __len__(self):
        return len(self.Y_data)

    # 인덱스를 기반으로 데이터세트로부터 샘플을 가져오는 함수 
    def __getitem__(self, idx):            

        # 데이터 지정
        image = self.X_data[idx]
        label = self.Y_data[idx]
        
        # 입력 데이터 전처리
        if self.transform:
            image = self.transform(image)

        # 출력 데이터 전처리
        if self.target_transform:
            label = self.target_transform(label)

        return image, label

In [4]:
import torchvision.transforms as transforms

train_path = "dataset/fashion-mnist_train.csv"
test_path = "dataset/fashion-mnist_test.csv"

# transforms.ToTensor() : PIL이미지,Numpy 배열 --> Pytorch image tensor로 변환 (배열구조 변경 및 scale 변환됨)
train_dataset = CustomDataset(train_path,
                              transform=transforms.Compose([transforms.ToTensor()]))


test_dataset = CustomDataset(test_path,
                             transform=transforms.Compose([transforms.ToTensor()]))

In [5]:
# "생성한 데이터세트 클래스"를 상속받은 "객체"를 입력으로 받음
# 데이터세트에서 데이터를 iterable 하게 가져옴
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)

#### 2. 모델구성

torch.nn: 신경망 구축을 위한 데이터 구조나 레이어 등의 라이브러리

In [6]:
import torch.nn as nn
class NN(nn.Module):
    def __init__(self):
        super(NN,self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
        )
    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [7]:
# 정의한 모델 클래스를 gpu로 지정후 인스턴스 생성
model = NN().to(device)
print(model)

NN(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


#### 3. 하이퍼파라미터 & LOSS & Optimizer 지정

In [30]:
def train_loop(dataloader,model,loss_fn,optimizer):
    
    # 입력 데이터세트의 총 길이 
    size = len(dataloader.dataset) # 60000

    # 배치 총 개수 (60000/64 = 938) 
    num_batches = len(dataloader)
    
    # train_loop 
    for batch, (X,y) in enumerate(dataloader): 

        # 타입 & device 변환
        X = X.type(torch.float32).to(device)
        y = y.to(device)

        # 모델 예측
        pred = model(X)
        
        # Loss 계산
        loss = loss_fn(pred,y)

        # Backpropagation
        optimizer.zero_grad() # 이전 루프에서 각 파라미터들에 저장된 .grad를 초기화
        loss.backward() # 각 파라미터들의 .grad값에 변화정도가 저장이 됨.
        optimizer.step() # 최적화함수에 맞게, 각 파라미터 업데이트

        # log
        if batch % num_batches == 0:
            loss = loss.item() # item()을 통해, 변수에서 값만 가져옴
            current = batch*len(X)
            print(f"Train loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader,model,loss_fn):

    # 입력 데이터세트의 총 길이
    size = len(dataloader.dataset) # 10000
    
    # 배치 총 개수 (10000/64 = 157)
    num_batches = len(dataloader)

    test_loss = 0
    correct = 0

    # 테스트 단계이므로 gradient 옵션 해제
    with torch.no_grad():
        for X,y in dataloader:
            
            # 타입통일 & device 변환
            X = X.type(torch.float32).to(device)
            y = y.to(device)

            # 모델 예측
            pred = model(X)

            # Loss 계산
            loss = loss_fn(pred,y)

            # LOSS / CORRECT 
            test_loss += loss.item() # item()을 통해, 변수에서 값만 가져옴
            correct_cnt = (pred.argmax(axis=1)==y).type(torch.float).sum().item()
            correct += correct_cnt
    
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [31]:
learning_rate = 1e-3
epochs = 30
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("학습완료")

Epoch 1
-------------------------------
Train loss: 0.136297  [    0/60000]
Test Error: 
 Accuracy: 89.6%, Avg loss: 0.363977 

Epoch 2
-------------------------------
Train loss: 0.153474  [    0/60000]
Test Error: 
 Accuracy: 88.8%, Avg loss: 0.378791 

Epoch 3
-------------------------------
Train loss: 0.104953  [    0/60000]
Test Error: 
 Accuracy: 88.7%, Avg loss: 0.393948 

Epoch 4
-------------------------------
Train loss: 0.089394  [    0/60000]
Test Error: 
 Accuracy: 89.5%, Avg loss: 0.366364 

Epoch 5
-------------------------------
Train loss: 0.049651  [    0/60000]
Test Error: 
 Accuracy: 87.3%, Avg loss: 0.466652 

Epoch 6
-------------------------------
Train loss: 0.157366  [    0/60000]
Test Error: 
 Accuracy: 88.5%, Avg loss: 0.409780 

Epoch 7
-------------------------------
Train loss: 0.089616  [    0/60000]
Test Error: 
 Accuracy: 89.5%, Avg loss: 0.377564 

Epoch 8
-------------------------------
Train loss: 0.091857  [    0/60000]
Test Error: 
 Accuracy: 89.5

#### 4. 모델 저장

In [38]:
# 학습된 가중치 - model.state_dict() 로 확인가능
model.state_dict()

OrderedDict([('linear_relu_stack.0.weight',
              tensor([[-0.0004, -0.0104,  0.0025,  ..., -0.0168,  0.0240, -0.0120],
                      [-0.0221, -0.0214, -0.0240,  ...,  0.0329, -0.0115, -0.0099],
                      [ 0.0040,  0.0240, -0.0281,  ..., -0.0088,  0.0015, -0.0051],
                      ...,
                      [ 0.0048, -0.0296, -0.0335,  ...,  0.0081,  0.0206,  0.0207],
                      [ 0.0229,  0.0224, -0.0221,  ...,  0.0150, -0.0139,  0.0048],
                      [ 0.0312, -0.0229, -0.0215,  ..., -0.0234, -0.0288,  0.0225]],
                     device='cuda:0')),
             ('linear_relu_stack.0.bias',
              tensor([-1.8264e-02, -3.4706e-02,  3.4503e-02,  1.5933e-02,  3.3503e-02,
                      -1.8268e-02, -1.7407e-02,  8.0160e-03, -3.2167e-02, -7.5695e-03,
                       4.2844e-03,  2.8266e-02,  8.4228e-03,  3.5166e-02,  1.3006e-02,
                       1.3063e-02,  8.7463e-04, -4.6216e-03,  4.1512e-03,  1.6867

가중치저장

In [35]:
torch.save(model.state_dict(),'checkpoints/NNtest.pth')

가중치로드

In [37]:
model = NN().to(device)
model.load_state_dict(torch.load('checkpoints/NNtest.pth'))

<All keys matched successfully>